In [1]:
%load_ext autoreload
%autoreload 2
import sys
# map folder using sys
#sys.path.append('../')
import pandas as pd
from galgoz import DATA_FOLDER
from galgoz import Galgoz
from scipy import signal
import numpy as np
from plotly import graph_objects as go
from galgoz.indicators.trend import SG
from whittaker_eilers import WhittakerSmoother
from plotly.subplots import make_subplots

gz=Galgoz()

In [2]:
filename = DATA_FOLDER / "GBP_JPY_H1.pkl"
data = pd.read_pickle(filename)
data["time"] = pd.to_datetime(data["time"])
data.set_index("time", inplace=True)

In [3]:
df_an = data.iloc[-1000:].copy()
sg_trend1 = SG(df_an["mid_c"]).output
sg_trend2 = SG(df_an["mid_c"].iloc[:-200]).output
sg_trend3 = SG(df_an["mid_c"].iloc[:-500]).output

wes = WhittakerSmoother(lmbda=1000000, order=2, data_length = len(df_an))
optimal_smooth = wes.smooth_optimal(df_an["mid_c"].values)
print("Optimal lambda: {}".format(optimal_smooth.get_optimal().get_lambda()))
wes_trend1 = wes.smooth(df_an["mid_c"])

# Plot the mid_c price as a line
df_plot = df_an.copy()
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=('Savitzky-Golay Filter', 'Volume'))

# Compare diffrent sg trends with less data
fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot["mid_c"], mode='lines', name='Mid Close', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_plot.index, y=sg_trend1, mode='lines', name='Trend SG', line=dict(color='green')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_plot.index[:-100], y=sg_trend2, mode='lines', name='Trend SG100', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=df_plot.index[:-200], y=sg_trend3, mode='lines', name='Trend SG200', line=dict(color='black')), row=1, col=1)

# Update layout to include the new subplot
fig.update_layout(
    height=1200,
    showlegend=True
)

num_ticks = len(df_an) // 10  # Show 10% of the ticks
fig.update_xaxes(tickvals=df_an.index[::num_ticks])
fig.update_layout(xaxis_rangeslider_visible=False, height=800)
fig.show()

Optimal lambda: 0.31622776601683794
